# 实践：UNet论文中的经典任务——医学图像分割

In [1]:
# 加载数据
import torch
import cv2
import os
import glob
from torch.utils.data import Dataset
import random

class ISBI_Loader(Dataset):
    def __init__(self, data_path):
        # 读取所有data_path下的图片
        self.data_path = data_path
        self.imgs_path = glob.glob(os.path.join(data_path, 'image/*.png'))

    def augment(self, image, flipCode):
        # 使用cv2.flip进行数据增强(使数据变多)
        # flipCode为1水平翻转，0垂直翻转， -1水平+垂直
        flip = cv2.flip(image, flipCode)
        return flip
    
    def __getitem__(self, index):
        image_path = self.imgs_path[index]
        label_path = image_path.replace('image', 'label')
        
        image = cv2.imread(image_path)
        label = cv2.imread(label_path)
        # 将数据转为单通道的图片
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        label = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)
        image = image.reshape(1, image.shape[0], image.shape[1])
        label = label.reshape(1, label.shape[0], label.shape[1])
        # 处理标签，范围缩至0-1
        if label.max()>1:
            label = label/255
        # 随机进行数据增强，为2时不做处理
        flipCode = random.choice([-1,0,1,2])
        if flipCode !=2:
            image = self.augment(image, flipCode)
            label = self.augment(label, flipCode)
        return image, label
    
    def __len__(self):
        return len(self.imgs_path)

isbi_dataset = ISBI_Loader(r"D:\_workPlace\Book\AI\CV\UNet\data\train")#路径必须是全英文
print("数据个数：", len(isbi_dataset))
train_loader = torch.utils.data.DataLoader(dataset=isbi_dataset,
                                           batch_size=2,
                                           shuffle=True)
for image, label in train_loader:
    print(image.shape)

数据个数： 30
torch.Size([2, 1, 512, 512])
torch.Size([2, 1, 512, 512])
torch.Size([2, 1, 512, 512])
torch.Size([2, 1, 512, 512])
torch.Size([2, 1, 512, 512])
torch.Size([2, 1, 512, 512])
torch.Size([2, 1, 512, 512])
torch.Size([2, 1, 512, 512])
torch.Size([2, 1, 512, 512])
torch.Size([2, 1, 512, 512])
torch.Size([2, 1, 512, 512])
torch.Size([2, 1, 512, 512])
torch.Size([2, 1, 512, 512])
torch.Size([2, 1, 512, 512])
torch.Size([2, 1, 512, 512])


In [2]:
# UNet模型，相比UNet文件中，对它进行了微调，使得网络的输出尺寸刚好等于图片的输入尺寸
import torch.nn as nn
import torch.nn.functional as F

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.double_conv(x)
    
class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )
    def forward(self, x):
        return self.maxpool_conv(x)

class Up(nn.Module):
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channels//2, in_channels//2, kernel_size=2,stride=2)
        self.conv = DoubleConv(in_channels, out_channels)
    def forward(self, x1, x2):
        x1 = self.up(x1)
        # 特征的融合
        diffY = torch.tensor([x2.size()[2]-x1.size()[2]])
        diffX = torch.tensor([x2.size()[3]-x1.size()[3]])
        # 填充
        x1 = F.pad(x1, [diffX//2, diffX-diffX//2,
                        diffY//2, diffY-diffY//2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        
    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear
        
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)
        self.up1 = Up(1024, 256, bilinear)
        self.up2 = Up(512, 128, bilinear)
        self.up3 = Up(256, 64, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)
    
    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [3]:
# 训练
from torch import optim
def train_net(net, device, data_path, epochs=40, batch_size=1, lr=0.00001):
    # 加载训练集
    isbi_dataset = ISBI_Loader(data_path)
    train_loader = torch.utils.data.DataLoader(dataset=isbi_dataset,
                                               batch_size=2,
                                               shuffle=True)
    # 定义RMSprop算法
    optimizer = optim.RMSprop(net.parameters(), lr=lr, weight_decay=1e-8, momentum=0.9)# weight_decay和momentum的详细介绍https://blog.csdn.net/xuxiatian/article/details/72771609?locationNum=7&fps=1
    # 定义Loss算法 BCEWithLogitsLoss比较高级的自适应优化算法
    criterion = nn.BCEWithLogitsLoss()
    # best_loss统计，初始化为正无穷
    best_loss = float('inf')
    # 训练epochs次
    for epoch in range(epochs):
        net.train()
        print('epoch ', epoch)
        for image, label in train_loader:
            optimizer.zero_grad()
            # 将数据拷贝到device中
            image = image.to(device=device, dtype=torch.float32)
            label = label.to(device=device, dtype=torch.float32)
            # 使用网络参数，输出预测结果
            pred = net(image)
            loss = criterion(pred, label)
            print('Loss/train', loss.item())
            # 保存loss值最小的网络参数
            if loss<best_loss:
                best_loss = loss
                torch.save(net.state_dict(), r'D:\_workPlace\Book\AI\CV\UNet\best_model.pth')
            #更新参数
            loss.backward()
            optimizer.step()

In [9]:
# 训练
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = UNet(n_channels=1, n_classes=1)
net.to(device=device)
data_path=r"D:\_workPlace\Book\AI\CV\UNet\data\train"
train_net(net, device, data_path)

Loss/train 0.844826877117157
Loss/train 0.7857764959335327
Loss/train 0.7505689263343811
Loss/train 0.7023769617080688
Loss/train 0.7376537919044495
Loss/train 0.6869902014732361
Loss/train 0.6566864848136902
Loss/train 0.6249339580535889
Loss/train 0.6147798299789429
Loss/train 0.6211022734642029
Loss/train 0.6024956703186035
Loss/train 0.5652669668197632
Loss/train 0.5660802721977234
Loss/train 0.5444195866584778
Loss/train 0.5884251594543457
Loss/train 0.5384031534194946
Loss/train 0.5142563581466675
Loss/train 0.4962066411972046
Loss/train 0.491624116897583
Loss/train 0.464009553194046
Loss/train 0.4441261887550354
Loss/train 0.46011587977409363
Loss/train 0.4961172640323639
Loss/train 0.4512881338596344
Loss/train 0.4315793514251709
Loss/train 0.4352658689022064
Loss/train 0.4426954388618469
Loss/train 0.43671542406082153
Loss/train 0.43543702363967896
Loss/train 0.4104059934616089
Loss/train 0.41676610708236694
Loss/train 0.4256676435470581
Loss/train 0.41161826252937317
Loss/tra

KeyboardInterrupt: 

In [5]:
# 训练时间有点长，好在中途存了模型，大概还剩20epoch，可以明天重新加载模型。需要在上述代码中做修改。
# 继续训练模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = UNet(n_channels=1, n_classes=1)
net.to(device=device)
net.load_state_dict(torch.load(r'D:\_workPlace\Book\AI\CV\UNet\best_model.pth', map_location=device))
data_path=r"D:\_workPlace\Book\AI\CV\UNet\data\train"
train_net(net, device, data_path, epochs=20)

epoch  0
Loss/train 0.2153027057647705
Loss/train 0.2036285400390625
Loss/train 0.2252168208360672
Loss/train 0.22698403894901276
Loss/train 0.23495785892009735
Loss/train 0.22438110411167145
Loss/train 0.22229404747486115
Loss/train 0.23490528762340546
Loss/train 0.232426255941391
Loss/train 0.2421875149011612
Loss/train 0.24110187590122223
Loss/train 0.2219175547361374
Loss/train 0.22014302015304565
Loss/train 0.225480318069458
Loss/train 0.2266569435596466
epoch  1
Loss/train 0.2102348953485489
Loss/train 0.21975284814834595
Loss/train 0.21820513904094696
Loss/train 0.23151665925979614
Loss/train 0.2126106172800064
Loss/train 0.21045231819152832
Loss/train 0.2308168113231659
Loss/train 0.21440434455871582
Loss/train 0.21008555591106415
Loss/train 0.20876458287239075
Loss/train 0.23725970089435577
Loss/train 0.21021120250225067
Loss/train 0.21051812171936035
Loss/train 0.2007526010274887
Loss/train 0.21985162794589996
epoch  2
Loss/train 0.22633305191993713
Loss/train 0.2322261780500

KeyboardInterrupt: 

In [9]:
import numpy as np
# 测试
net.load_state_dict(torch.load(r'D:\_workPlace\Book\AI\CV\UNet\best_model.pth', map_location=device))
# 测试模式
net.eval()
tests_path = glob.glob('D:/_workPlace/Book/AI/CV/UNet/data/test/*.png')
for test_path in tests_path:
    print(test_path)
    save_res_path = test_path.split('.')[0]+'_res.png'
    img = cv2.imread(test_path)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 转为batch为1，通道为1，大小为512*512的数组
    img = img.reshape(1, 1, img.shape[0], img.shape[1])
    img_tensor = torch.from_numpy(img)
    img_tensor = img_tensor.to(device=device, dtype=torch.float32)
    pred = net(img_tensor)
    pred=np.array(pred.data.cpu()[0][0])# 提取结果
    pred[pred>=0.5]=255
    pred[pred<0.5]=0
    cv2.imwrite(save_res_path, pred)

D:/_workPlace/Book/AI/CV/UNet/data/test\0.png
D:/_workPlace/Book/AI/CV/UNet/data/test\1.png
D:/_workPlace/Book/AI/CV/UNet/data/test\10.png
D:/_workPlace/Book/AI/CV/UNet/data/test\11.png
D:/_workPlace/Book/AI/CV/UNet/data/test\12.png
D:/_workPlace/Book/AI/CV/UNet/data/test\13.png
D:/_workPlace/Book/AI/CV/UNet/data/test\14.png
D:/_workPlace/Book/AI/CV/UNet/data/test\15.png
D:/_workPlace/Book/AI/CV/UNet/data/test\16.png
D:/_workPlace/Book/AI/CV/UNet/data/test\17.png
D:/_workPlace/Book/AI/CV/UNet/data/test\18.png
D:/_workPlace/Book/AI/CV/UNet/data/test\19.png
D:/_workPlace/Book/AI/CV/UNet/data/test\2.png
D:/_workPlace/Book/AI/CV/UNet/data/test\20.png
D:/_workPlace/Book/AI/CV/UNet/data/test\21.png
D:/_workPlace/Book/AI/CV/UNet/data/test\22.png
D:/_workPlace/Book/AI/CV/UNet/data/test\23.png
D:/_workPlace/Book/AI/CV/UNet/data/test\24.png
D:/_workPlace/Book/AI/CV/UNet/data/test\25.png
D:/_workPlace/Book/AI/CV/UNet/data/test\26.png
D:/_workPlace/Book/AI/CV/UNet/data/test\27.png
D:/_workPlace/Bo